In [23]:
import google.generativeai as genai

genai.configure(api_key="your-google-ai-api-key")

models = genai.list_models()

for model in models:
    print(model.name)

models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thin

In [12]:
import os
os.environ['GOOGLE_API_KEY'] = 'your-google-ai-api-key'

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate

llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-pro", temperature=0.8)

prompt = PromptTemplate.from_template("I want to open an {restro} restrobar, suggest me a fancy name for it and a {cuisine} dish.")
# response = llm.invoke(prompt)

chain = prompt | llm

# Run the chain
response = chain.invoke({"restro":"Arabic", "cuisine":"Chinese"})

# Print the result
print(response.content)

Of course! Opening an Arabic restrobar is a fantastic idea. The concept allows for a rich tapestry of culture, flavor, and modern style.

Here are some suggestions for a fancy name and a Chinese dish designed to fit seamlessly into the menu.

---

### Fancy Name Suggestions for an Arabic Restrobar

The key is to choose a name that is evocative, elegant, and easy for people to remember and pronounce.

**Option 1: Poetic & Romantic**
*   **Layla's Lounge** (ليلى) - "Layla" means "night" in Arabic. It's a classic, beautiful name that evokes romance, starry desert nights, and the stories of *One Thousand and One Nights*. The word "Lounge" immediately signals a chic, relaxed bar atmosphere.

**Option 2: Modern & Chic**
*   **Arabesque** - This single word is sophisticated and stylish. It refers to the intricate, flowing, and ornate patterns of Islamic art. It suggests a place of beauty, detail, and artistry, both in its decor and its food and drink.

**Option 3: Luxurious & Grand**
*   **Th

In [5]:
import os
os.environ['GOOGLE_API_KEY'] = 'your-google-ai-api-key'

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableMap

llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-pro", temperature=0.8)

# 1) Generate restrobar name
prompt_name = PromptTemplate.from_template("Suggest a fancy {cuisine} restrobar name.")
name_chain = prompt_name | llm

# 2) Function to generate dietary-based prompt
def make_diet_prompt(inputs):
    cuisine = inputs["cuisine"]
    diet = inputs["diet"]
    restro = inputs["restro_name"]
    
    if diet == "vegan":
        prompt_text = f"Suggest 5 best-selling vegan {cuisine} dishes for the restrobar named {restro}."
    elif diet == "non-vegan":
        prompt_text = f"Suggest 5 best-selling non-vegan {cuisine} dishes for the restrobar named {restro}."
    else:
        prompt_text = f"Suggest 5 best-selling {cuisine} dishes for the restrobar named {restro} that suit both vegan and non-vegan preferences."
    
    return {"prompt": prompt_text}

diet_router = RunnableLambda(make_diet_prompt)
prompt_dishes = PromptTemplate.from_template("{prompt}")
dishes_chain = diet_router | prompt_dishes | llm

# Full pipeline using RunnableLambda instead of RunnableMap
full_chain = (
    RunnableLambda(lambda x: {"cuisine": x["cuisine"], "diet": x["diet"]})
    | RunnableLambda(lambda x: {
        "cuisine": x["cuisine"],
        "diet": x["diet"],
        "restro_name": name_chain.invoke({"cuisine": x["cuisine"]}).content
    })
    | RunnableLambda(lambda x: {
        "restro_name": x["restro_name"],
        "dishes": dishes_chain.invoke({
            "restro_name": x["restro_name"],
            "cuisine": x["cuisine"],
            "diet": x["diet"]
        }).content
    })
)

# Run the chain
input_data = {"cuisine": "Indian", "diet": "vegan"}
result = full_chain.invoke(input_data)

# Output
print("Restaurant name: ", result["restro_name"])
print("\nDiet-based dishes: \n", result["dishes"])

Restaurant name:  Of course! Here is a list of fancy Indian restrobar names, categorized by the vibe they evoke.

### Modern & Chic
These names sound sophisticated, contemporary, and are perfect for a trendy, urban location.

*   **The Tamra Room** (Tamra means copper in Sanskrit, evoking copper mugs for cocktails and traditional cookware.)
*   **Masala & Tonic** (A playful, direct name that clearly communicates "Indian" + "Bar".)
*   **Cinnamon Social** (Suggests warmth, spice, and a place to gather.)
*   **The Vermilion** (A beautiful, deep red color associated with India. It's elegant and memorable.)
*   **Aura by [Chef's Name]** (Puts the focus on a high-end, chef-driven experience.)
*   **Kamaa** (Means "art" or "work" in Hindi, suggesting culinary artistry.)

### Royal & Heritage
These names evoke the grandeur of Indian palaces, maharajas, and old-world luxury.

*   **The Darbar Hall** (Darbar means "royal court," suggesting a grand, opulent experience.)
*   **Raj Vilas** (Litera